# Feature interactions

With the two functions in this module it is possible to generate interactions between numerical and categorical variables without losing their names in the process.

[Data](https://www.kaggle.com/jsphyg/weather-dataset-rattle-package)<br>
[Module](https://github.com/abreukuse/ml_utilities/blob/master/feature_interactions.py) 

In [ ]:
# Feature Engine is a very nice, easy to use and well documented library 
# that helps a lot in the task of building pipelines for data preprocessing.
# https://feature-engine.readthedocs.io/en/latest/quickstart.html
!pip install feature-engine

In [2]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from feature_engine.wrappers import SklearnTransformerWrapper

pd.set_option('display.max_columns', 3000)

from feature_interactions import numerical_interactions, categorical_interactions

import os
os.environ['KAGGLE_USERNAME'] = 'kaggle_username'
os.environ['KAGGLE_KEY'] = 'kaggle_api_key'

In [ ]:
!kaggle datasets download -d jsphyg/weather-dataset-rattle-package
!unzip 'weather-dataset-rattle-package.zip'

In [4]:
data = pd.read_csv('weatherAUS.csv')
data.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [5]:
numerical = data.select_dtypes('number')
numerical.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
0,13.4,22.9,0.6,NaN,NaN,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8
1,7.4,25.1,0.0,NaN,NaN,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3
2,12.9,25.7,0.0,NaN,NaN,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2
3,9.2,28.0,0.0,NaN,NaN,24.0,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5
4,17.5,32.3,1.0,NaN,NaN,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7


In [6]:
categorical = data.select_dtypes('object').drop(columns=['Date','Location','RainTomorrow'])
categorical.head()

,WindGustDir,WindDir9am,WindDir3pm,RainToday
0,W,W,WNW,No
1,WNW,NNW,WSW,No
2,WSW,W,WSW,No
3,NE,SE,E,No
4,W,ENE,NW,No


In [7]:
# Impute missing values
numerical = SklearnTransformerWrapper(transformer=SimpleImputer(strategy='median')).fit_transform(numerical)
categorical = SklearnTransformerWrapper(transformer=SimpleImputer(strategy='most_frequent')).fit_transform(categorical)

In [8]:
numerical.isnull().sum().sum()

0

In [9]:
categorical.isnull().sum().sum()

0

In [10]:
numerical = numerical_interactions(X=numerical, variables='all', degree=2, interaction_only=False)

In [11]:
numerical.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,MinTemp^2,MinTemp MaxTemp,MinTemp Rainfall,MinTemp Evaporation,MinTemp Sunshine,MinTemp WindGustSpeed,MinTemp WindSpeed9am,MinTemp WindSpeed3pm,MinTemp Humidity9am,MinTemp Humidity3pm,MinTemp Pressure9am,MinTemp Pressure3pm,MinTemp Cloud9am,MinTemp Cloud3pm,MinTemp Temp9am,MinTemp Temp3pm,MaxTemp^2,MaxTemp Rainfall,MaxTemp Evaporation,MaxTemp Sunshine,MaxTemp WindGustSpeed,MaxTemp WindSpeed9am,MaxTemp WindSpeed3pm,MaxTemp Humidity9am,MaxTemp Humidity3pm,MaxTemp Pressure9am,MaxTemp Pressure3pm,MaxTemp Cloud9am,MaxTemp Cloud3pm,MaxTemp Temp9am,MaxTemp Temp3pm,Rainfall^2,Rainfall Evaporation,Rainfall Sunshine,Rainfall WindGustSpeed,Rainfall WindSpeed9am,Rainfall WindSpeed3pm,Rainfall Humidity9am,Rainfall Humidity3pm,Rainfall Pressure9am,Rainfall Pressure3pm,Rainfall Cloud9am,Rainfall Cloud3pm,Rainfall Temp9am,Rainfall Temp3pm,Evaporation^2,Evaporation Sunshine,Evaporation WindGustSpeed,Evaporation WindSpeed9am,Evaporation WindSpeed3pm,Evaporation Humidity9am,Evaporation Humidity3pm,Evaporation Pressure9am,Evaporation Pressure3pm,Evaporation Cloud9am,Evaporation Cloud3pm,Evaporation Temp9am,Evaporation Temp3pm,Sunshine^2,Sunshine WindGustSpeed,Sunshine WindSpeed9am,Sunshine WindSpeed3pm,Sunshine Humidity9am,Sunshine Humidity3pm,Sunshine Pressure9am,Sunshine Pressure3pm,Sunshine Cloud9am,Sunshine Cloud3pm,Sunshine Temp9am,Sunshine Temp3pm,WindGustSpeed^2,WindGustSpeed WindSpeed9am,WindGustSpeed WindSpeed3pm,WindGustSpeed Humidity9am,WindGustSpeed Humidity3pm,WindGustSpeed Pressure9am,WindGustSpeed Pressure3pm,WindGustSpeed Cloud9am,WindGustSpeed Cloud3pm,WindGustSpeed Temp9am,WindGustSpeed Temp3pm,WindSpeed9am^2,WindSpeed9am WindSpeed3pm,WindSpeed9am Humidity9am,WindSpeed9am Humidity3pm,WindSpeed9am Pressure9am,WindSpeed9am Pressure3pm,WindSpeed9am Cloud9am,WindSpeed9am Cloud3pm,WindSpeed9am Temp9am,WindSpeed9am Temp3pm,WindSpeed3pm^2,WindSpeed3pm Humidity9am,WindSpeed3pm Humidity3pm,WindSpeed3pm Pressure9am,WindSpeed3pm Pressure3pm,WindSpeed3pm Cloud9am,WindSpeed3pm Cloud3pm,WindSpeed3pm Temp9am,WindSpeed3pm Temp3pm,Humidity9am^2,Humidity9am Humidity3pm,Humidity9am Pressure9am,Humidity9am Pressure3pm,Humidity9am Cloud9am,Humidity9am Cloud3pm,Humidity9am Temp9am,Humidity9am Temp3pm,Humidity3pm^2,Humidity3pm Pressure9am,Humidity3pm Pressure3pm,Humidity3pm Cloud9am,Humidity3pm Cloud3pm,Humidity3pm Temp9am,Humidity3pm Temp3pm,Pressure9am^2,Pressure9am Pressure3pm,Pressure9am Cloud9am,Pressure9am Cloud3pm,Pressure9am Temp9am,Pressure9am Temp3pm,Pressure3pm^2,Pressure3pm Cloud9am,Pressure3pm Cloud3pm,Pressure3pm Temp9am,Pressure3pm Temp3pm,Cloud9am^2,Cloud9am Cloud3pm,Cloud9am Temp9am,Cloud9am Temp3pm,Cloud3pm^2,Cloud3pm Temp9am,Cloud3pm Temp3pm,Temp9am^2,Temp9am Temp3pm,Temp3pm^2
0,13.4,22.9,0.6,4.8,8.4,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,5.0,16.9,21.8,179.56,306.86,8.04,64.32,112.56,589.6,268.0,321.6,951.4,294.8,13503.18,13495.14,107.2,67.0,226.46,292.12,524.41,13.74,109.92,192.36,1007.6,458.0,549.6,1625.9,503.8,23076.33,23062.59,183.2,114.5,387.01,499.22,0.36,2.88,5.04,26.4,12.0,14.4,42.6,13.2,604.62,604.26,4.8,3.0,10.14,13.08,23.04,40.32,211.2,96.0,115.2,340.8,105.6,4836.96,4834.08,38.4,24.0,81.12,104.64,70.56,369.6,168.0,201.6,596.4,184.8,8464.68,8459.64,67.2,42.0,141.96,183.12,1936.0,880.0,1056.0,3124.0,968.0,44338.8,44312.4,352.0,220.0,743.6,959.2,400.0,480.0,1420.0,440.0,20154.0,20142.0,160.0,100.0,338.0,436.0,576.0,1704.0,528.0,24184.8,24170.4,192.0,120.0,405.6,523.2,5041.0,1562.0,71546.7,71504.1,568.0,355.0,1199.9,1547.8,484.0,22169.4,22156.2,176.0,110.0,371.8,479.6,1015459.29,1014854.67,8061.6,5038.5,17030.13,21967.86,1014250.41,8056.8,5035.5,17019.99,21954.78,64.0,40.0,135.2,174.4,25.0,84.5,109.0,285.61,368.42,475.24
1,7.4,25.1,0.0,4.8,8.4,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,5.0,5.0,17.2,24.3,54.76,185.74,0.00,35.52,62.16,325.6,29.6,162.8,325.6,185.0

In [12]:
categorical = categorical_interactions(X=categorical, variables='all', order=4)

In [13]:
categorical.head()

,WindGustDir,WindDir9am,WindDir3pm,RainToday,WindGustDir-WindDir9am,WindGustDir-WindDir3pm,WindGustDir-RainToday,WindDir9am-WindDir3pm,WindDir9am-RainToday,WindDir3pm-RainToday,WindGustDir-WindDir9am-WindDir3pm,WindGustDir-WindDir9am-RainToday,WindGustDir-WindDir3pm-RainToday,WindDir9am-WindDir3pm-RainToday,WindGustDir-WindDir9am-WindDir3pm-RainToday
0,W,W,WNW,No,W-W,W-WNW,W-No,W-WNW,W-No,WNW-No,W-W-WNW,W-W-No,W-WNW-No,W-WNW-No,W-W-WNW-No
1,WNW,NNW,WSW,No,WNW-NNW,WNW-WSW,WNW-No,NNW-WSW,NNW-No,WSW-No,WNW-NNW-WSW,WNW-NNW-No,WNW-WSW-No,NNW-WSW-No,WNW-NNW-WSW-No
2,WSW,W,WSW,No,WSW-W,WSW-WSW,WSW-No,W-WSW,W-No,WSW-No,WSW-W-WSW,WSW-W-No,WSW-WSW-No,W-WSW-No,WSW-W-WSW-No
3,NE,SE,E,No,NE-SE,NE-E,NE-No,SE-E,SE-No,E-No,NE-SE-E,NE-SE-No,NE-E-No,SE-E-No,NE-SE-E-No
4,W,ENE,NW,No,W-ENE,W-NW,W-No,ENE-NW,ENE-No,NW-No,W-ENE-NW,W-ENE-No,W-NW-No,ENE-NW-No,W-ENE-NW-No


## Pipeline

In [14]:
VARIABLES = [
             # numerical
             'MinTemp',
             'MaxTemp',
             'Rainfall',
             'Evaporation',
             'Sunshine',
             'WindGustSpeed',
             'WindSpeed9am',
             'WindSpeed3pm',
             'Humidity9am',
             'Humidity3pm',
             'Pressure9am',
             'Pressure3pm',
             'Cloud9am',
             'Cloud3pm',
             'Temp9am',
             'Temp3pm',
             
             # categorical
             'WindGustDir',
             'WindDir9am',
             'WindDir3pm',
             'RainToday'
             ]

NUMERICAL = [
             'MinTemp',
             'MaxTemp',
             'Rainfall',
             'Evaporation',
             'Sunshine',
             'WindGustSpeed',
             'WindSpeed9am',
             'WindSpeed3pm',
             'Humidity9am',
             'Humidity3pm',
             'Pressure9am',
             'Pressure3pm',
             'Cloud9am',
             'Cloud3pm',
             'Temp9am',
             'Temp3pm'
             ]

CATEGORICAL = [
               'WindGustDir',
               'WindDir9am',
               'WindDir3pm',
               'RainToday'
               ]             

def select(X, variables):
    """Select just a few variables for the demonstration."""
    X = X[variables].copy()
    return X

pipeline = make_pipeline(

    # Selection of varibles
    FunctionTransformer(select, 
                        kw_args={'variables': VARIABLES}
                        ),

    # Missing data imputation for numerical variables
    SklearnTransformerWrapper(variables=NUMERICAL,
                              transformer=SimpleImputer(strategy='median')
                              ),

    # Missing data imputation for categorical variables
    SklearnTransformerWrapper(variables=CATEGORICAL,
                              transformer=SimpleImputer(strategy='most_frequent')
                              ),

    # Creation of numerical interactions features
    FunctionTransformer(numerical_interactions, 
                        kw_args={'variables': NUMERICAL,
                                 'degree': 2,
                                 'interaction_only': True
                                 }
                        ),

    # Creation of categorical interactions features
    FunctionTransformer(categorical_interactions,
                        kw_args={'variables': CATEGORICAL,
                                 'order': 4
                                 }
                        )                         
)

In [15]:
# There are missing data in the target
data = data.loc[~data['RainTomorrow'].isnull(), :].copy()

# Date/time conversion
data['Date'] = pd.to_datetime(data['Date'])

In [16]:
# Simple split
train = data.query('Date < "2015-01-01"').copy()
validation = data.query('Date >= "2015-01-01"').copy()

In [17]:
X_train = pipeline.fit_transform(train)

In [18]:
X_train.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,WindGustDir,WindDir9am,WindDir3pm,RainToday,MinTemp MaxTemp,MinTemp Rainfall,MinTemp Evaporation,MinTemp Sunshine,MinTemp WindGustSpeed,MinTemp WindSpeed9am,MinTemp WindSpeed3pm,MinTemp Humidity9am,MinTemp Humidity3pm,MinTemp Pressure9am,MinTemp Pressure3pm,MinTemp Cloud9am,MinTemp Cloud3pm,MinTemp Temp9am,MinTemp Temp3pm,MaxTemp Rainfall,MaxTemp Evaporation,MaxTemp Sunshine,MaxTemp WindGustSpeed,MaxTemp WindSpeed9am,MaxTemp WindSpeed3pm,MaxTemp Humidity9am,MaxTemp Humidity3pm,MaxTemp Pressure9am,MaxTemp Pressure3pm,MaxTemp Cloud9am,MaxTemp Cloud3pm,MaxTemp Temp9am,MaxTemp Temp3pm,Rainfall Evaporation,Rainfall Sunshine,Rainfall WindGustSpeed,Rainfall WindSpeed9am,Rainfall WindSpeed3pm,Rainfall Humidity9am,Rainfall Humidity3pm,Rainfall Pressure9am,Rainfall Pressure3pm,Rainfall Cloud9am,Rainfall Cloud3pm,Rainfall Temp9am,Rainfall Temp3pm,Evaporation Sunshine,Evaporation WindGustSpeed,Evaporation WindSpeed9am,Evaporation WindSpeed3pm,Evaporation Humidity9am,Evaporation Humidity3pm,Evaporation Pressure9am,Evaporation Pressure3pm,Evaporation Cloud9am,Evaporation Cloud3pm,Evaporation Temp9am,Evaporation Temp3pm,Sunshine WindGustSpeed,Sunshine WindSpeed9am,Sunshine WindSpeed3pm,Sunshine Humidity9am,Sunshine Humidity3pm,Sunshine Pressure9am,Sunshine Pressure3pm,Sunshine Cloud9am,Sunshine Cloud3pm,Sunshine Temp9am,Sunshine Temp3pm,WindGustSpeed WindSpeed9am,WindGustSpeed WindSpeed3pm,WindGustSpeed Humidity9am,WindGustSpeed Humidity3pm,WindGustSpeed Pressure9am,WindGustSpeed Pressure3pm,WindGustSpeed Cloud9am,WindGustSpeed Cloud3pm,WindGustSpeed Temp9am,WindGustSpeed Temp3pm,WindSpeed9am WindSpeed3pm,WindSpeed9am Humidity9am,WindSpeed9am Humidity3pm,WindSpeed9am Pressure9am,WindSpeed9am Pressure3pm,WindSpeed9am Cloud9am,WindSpeed9am Cloud3pm,WindSpeed9am Temp9am,WindSpeed9am Temp3pm,WindSpeed3pm Humidity9am,WindSpeed3pm Humidity3pm,WindSpeed3pm Pressure9am,WindSpeed3pm Pressure3pm,WindSpeed3pm Cloud9am,WindSpeed3pm Cloud3pm,WindSpeed3pm Temp9am,WindSpeed3pm Temp3pm,Humidity9am Humidity3pm,Humidity9am Pressure9am,Humidity9am Pressure3pm,Humidity9am Cloud9am,Humidity9am Cloud3pm,Humidity9am Temp9am,Humidity9am Temp3pm,Humidity3pm Pressure9am,Humidity3pm Pressure3pm,Humidity3pm Cloud9am,Humidity3pm Cloud3pm,Humidity3pm Temp9am,Humidity3pm Temp3pm,Pressure9am Pressure3pm,Pressure9am Cloud9am,Pressure9am Cloud3pm,Pressure9am Temp9am,Pressure9am Temp3pm,Pressure3pm Cloud9am,Pressure3pm Cloud3pm,Pressure3pm Temp9am,Pressure3pm Temp3pm,Cloud9am Cloud3pm,Cloud9am Temp9am,Cloud9am Temp3pm,Cloud3pm Temp9am,Cloud3pm Temp3pm,Temp9am Temp3pm,WindGustDir-WindDir9am,WindGustDir-WindDir3pm,WindGustDir-RainToday,WindDir9am-WindDir3pm,WindDir9am-RainToday,WindDir3pm-RainToday,WindGustDir-WindDir9am-WindDir3pm,WindGustDir-WindDir9am-RainToday,WindGustDir-WindDir3pm-RainToday,WindDir9am-WindDir3pm-RainToday,WindGustDir-WindDir9am-WindDir3pm-RainToday
0,13.4,22.9,0.6,4.6,8.4,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,5.0,16.9,21.8,W,W,WNW,No,306.86,8.04,61.64,112.56,589.6,268.0,321.6,951.4,294.8,13503.18,13495.14,107.2,67.0,226.46,292.12,13.74,105.34,192.36,1007.6,458.0,549.6,1625.9,503.8,23076.33,23062.59,183.2,114.5,387.01,499.22,2.76,5.04,26.4,12.0,14.4,42.6,13.2,604.62,604.26,4.8,3.0,10.14,13.08,38.64,202.4,92.0,110.4,326.6,101.2,4635.42,4632.66,36.8,23.0,77.74,100.28,369.6,168.0,201.6,596.4,184.8,8464.68,8459.64,67.2,42.0,141.96,183.12,880.0,1056.0,3124.0,968.0,44338.8,44312.4,352.0,220.0,743.6,959.2,480.0,1420.0,440.0,20154.0,20142.0,160.0,100.0,338.0,436.0,1704.0,528.0,24184.8,24170.4,192.0,120.0,405.6,523.2,1562.0,71546.7,71504.1,568.0,355.0,1199.9,1547.8,22169.4,22156.2,176.0,110.0,371.8,479.6,1014854.67,8061.6,5038.5,17030.13,21967.86,8056.8,5035.5,17019.99,21954.78,40.0,135.2,174.4,84.5,109.0,368.42,W-W,W-WNW,W-No,W-WNW,W-No,WNW-No,W-W-WNW,W-W-No,W-WNW-No,W-WNW-No,W-W-WNW-No
1,7.4,

In [19]:
X_validation = pipeline.transform(validation)

In [20]:
X_validation.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,WindGustDir,WindDir9am,WindDir3pm,RainToday,MinTemp MaxTemp,MinTemp Rainfall,MinTemp Evaporation,MinTemp Sunshine,MinTemp WindGustSpeed,MinTemp WindSpeed9am,MinTemp WindSpeed3pm,MinTemp Humidity9am,MinTemp Humidity3pm,MinTemp Pressure9am,MinTemp Pressure3pm,MinTemp Cloud9am,MinTemp Cloud3pm,MinTemp Temp9am,MinTemp Temp3pm,MaxTemp Rainfall,MaxTemp Evaporation,MaxTemp Sunshine,MaxTemp WindGustSpeed,MaxTemp WindSpeed9am,MaxTemp WindSpeed3pm,MaxTemp Humidity9am,MaxTemp Humidity3pm,MaxTemp Pressure9am,MaxTemp Pressure3pm,MaxTemp Cloud9am,MaxTemp Cloud3pm,MaxTemp Temp9am,MaxTemp Temp3pm,Rainfall Evaporation,Rainfall Sunshine,Rainfall WindGustSpeed,Rainfall WindSpeed9am,Rainfall WindSpeed3pm,Rainfall Humidity9am,Rainfall Humidity3pm,Rainfall Pressure9am,Rainfall Pressure3pm,Rainfall Cloud9am,Rainfall Cloud3pm,Rainfall Temp9am,Rainfall Temp3pm,Evaporation Sunshine,Evaporation WindGustSpeed,Evaporation WindSpeed9am,Evaporation WindSpeed3pm,Evaporation Humidity9am,Evaporation Humidity3pm,Evaporation Pressure9am,Evaporation Pressure3pm,Evaporation Cloud9am,Evaporation Cloud3pm,Evaporation Temp9am,Evaporation Temp3pm,Sunshine WindGustSpeed,Sunshine WindSpeed9am,Sunshine WindSpeed3pm,Sunshine Humidity9am,Sunshine Humidity3pm,Sunshine Pressure9am,Sunshine Pressure3pm,Sunshine Cloud9am,Sunshine Cloud3pm,Sunshine Temp9am,Sunshine Temp3pm,WindGustSpeed WindSpeed9am,WindGustSpeed WindSpeed3pm,WindGustSpeed Humidity9am,WindGustSpeed Humidity3pm,WindGustSpeed Pressure9am,WindGustSpeed Pressure3pm,WindGustSpeed Cloud9am,WindGustSpeed Cloud3pm,WindGustSpeed Temp9am,WindGustSpeed Temp3pm,WindSpeed9am WindSpeed3pm,WindSpeed9am Humidity9am,WindSpeed9am Humidity3pm,WindSpeed9am Pressure9am,WindSpeed9am Pressure3pm,WindSpeed9am Cloud9am,WindSpeed9am Cloud3pm,WindSpeed9am Temp9am,WindSpeed9am Temp3pm,WindSpeed3pm Humidity9am,WindSpeed3pm Humidity3pm,WindSpeed3pm Pressure9am,WindSpeed3pm Pressure3pm,WindSpeed3pm Cloud9am,WindSpeed3pm Cloud3pm,WindSpeed3pm Temp9am,WindSpeed3pm Temp3pm,Humidity9am Humidity3pm,Humidity9am Pressure9am,Humidity9am Pressure3pm,Humidity9am Cloud9am,Humidity9am Cloud3pm,Humidity9am Temp9am,Humidity9am Temp3pm,Humidity3pm Pressure9am,Humidity3pm Pressure3pm,Humidity3pm Cloud9am,Humidity3pm Cloud3pm,Humidity3pm Temp9am,Humidity3pm Temp3pm,Pressure9am Pressure3pm,Pressure9am Cloud9am,Pressure9am Cloud3pm,Pressure9am Temp9am,Pressure9am Temp3pm,Pressure3pm Cloud9am,Pressure3pm Cloud3pm,Pressure3pm Temp9am,Pressure3pm Temp3pm,Cloud9am Cloud3pm,Cloud9am Temp9am,Cloud9am Temp3pm,Cloud3pm Temp9am,Cloud3pm Temp3pm,Temp9am Temp3pm,WindGustDir-WindDir9am,WindGustDir-WindDir3pm,WindGustDir-RainToday,WindDir9am-WindDir3pm,WindDir9am-RainToday,WindDir3pm-RainToday,WindGustDir-WindDir9am-WindDir3pm,WindGustDir-WindDir9am-RainToday,WindGustDir-WindDir3pm-RainToday,WindDir9am-WindDir3pm-RainToday,WindGustDir-WindDir9am-WindDir3pm-RainToday
2133,11.4,33.5,0.0,4.6,8.4,30.0,7.0,11.0,45.0,14.0,1013.5,1011.0,5.0,5.0,21.0,32.7,WSW,ESE,W,No,381.90,0.0,52.44,95.76,342.0,79.8,125.4,513.0,159.6,11553.90,11525.40,57.0,57.0,239.40,372.78,0.0,154.10,281.40,1005.0,234.5,368.5,1507.5,469.0,33952.25,33868.50,167.5,167.5,703.50,1095.45,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38.64,138.0,32.2,50.6,207.0,64.4,4662.10,4650.60,23.0,23.0,96.60,150.42,252.0,58.8,92.4,378.0,117.6,8513.40,8492.40,42.0,42.0,176.40,274.68,210.0,330.0,1350.0,420.0,30405.0,30330.0,150.0,150.0,630.0,981.0,77.0,315.0,98.0,7094.5,7077.0,35.0,35.0,147.0,228.9,495.0,154.0,11148.5,11121.0,55.0,55.0,231.0,359.7,630.0,45607.5,45495.0,225.0,225.0,945.0,1471.5,14189.0,14154.0,70.0,70.0,294.0,457.8,1024648.50,5067.5,5067.5,21283.50,33141.45,5055.0,5055.0,21231.00,33059.70,25.0,105.0,163.5,105.0,163.5,686.70,WSW-ESE,WSW-W,WSW-No,ESE-W,ESE-No,W-No,WSW-ESE-W,WSW-ESE-No,WSW-W-No,ESE-W-No,WSW-ESE-W-No
2134,15.5,39.6,0.0,4.6,8.4,5

From here the preprocessing steps could go on and on with feature scaling, categorical encoding, so on and so forth.